In [5]:
import dhlab as dh
import pandas as pd
import dhlab.graph_networkx_louvain as gnl
import networkx as nx
from IPython.display import HTML

In [6]:
import igraph as ig
import leidenalg as la

from igraph import Graph
from leidenalg import find_partition
import numpy as np
import os

In [8]:
import conc_coll_corpus as co
import polnet_graphs as polg

In [9]:
from importlib import reload

In [10]:
zotero = co.zot()

In [11]:
G = polg.make_nx_graph(zotero)

## convert to igraph and use leiden algorithm

In [91]:
reload(polg)

<module 'polnet_graphs' from '/mnt/disk1/Github/polnet_plotly/polnet_graphs.py'>

In [12]:
h = polg.igraph_from_networkx(G)

In [13]:
degrees = polg.degree(h).set_index('vertex_id').to_dict()['degree']

In [14]:
zotero.columns

Index(['Key', 'Item Type', 'Publication Year', 'Author', 'Title',
       'Publication Title', 'ISBN', 'Departement', 'Utdanning', 'Reform',
       'Tematisk utvalg', 'Url', 'Abstract Note', 'Date', 'Date Added',
       'Date Modified', 'Access Date', 'Pages', 'Num Pages', 'Issue', 'Volume',
       'Number Of Volumes', 'Journal Abbreviation', 'Short Title', 'Series',
       'Series Number', 'Series Text', 'Series Title', 'Publisher', 'Place',
       'Language', 'Rights', 'Type2', 'Archive', 'Archive Location',
       'Library Catalog', 'Call Number', 'Extra', 'Notes', 'File Attachments',
       'Link Attachments', 'Manual Tags', 'Automatic Tags', 'Editor',
       'Series Editor', 'Translator', 'Contributor', 'Attorney Agent',
       'Book Author', 'Cast Member', 'Commenter', 'Composer', 'Cosponsor',
       'Counsel', 'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
       'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
       'Running Time', 'Scale', 'Medium', 'Artwo

In [17]:
years = zotero[['Key','Publication Year']].set_index('Key').to_dict()['Publication Year']

In [19]:
#years

In [20]:
for v in h.vs:
    v['centrality'] = degrees.get(v['name'], 0)
    v['year'] = years.get(v['name'], 1980)

In [154]:
#dict([(x[0], x[1]) for  x in polg.degree(h).set_index('vertex_id').to_records()])

In [21]:
parts = polg.make_clusters(h)

In [22]:
h.vs['community'] = parts.membership

In [111]:
#[vertex for vertex in h.vs]

In [23]:
import json

# Assuming 'h' is your igraph graph

# Extract nodes
nodes = [{"id": vertex['name'], "label": vertex['label'], 'year': vertex['year'], 'community': vertex['community'], 'centrality':int(vertex['centrality'])} for vertex in h.vs]

# Extract edges
links = [{"source": edge.source_vertex['name'], "target": edge.target_vertex['name']} for edge in h.es]

# Create a graph dictionary
graph = {"nodes": nodes, "links": links}

# Convert to JSON (if needed)
graph_json = json.dumps(graph)

# 'graph_json' is now ready to be used with D3.js


In [55]:
with open('polnet_dag.html', "w", encoding = "utf-8") as fp:
    fp.write(f"""
<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:xlink="http://www.w3.org/1999/xlink">
   <head>
      <meta charset="UTF-8">
      <title>D3 Force Layout</title>
      <style>
         text {{
         font-family: sans-serif;
         }}
      </style>
   </head>
   <body>
      <svg width="1800" height="2600">
       <defs>
           <marker id="arrowhead" markerWidth="10" markerHeight="7" 
  refX="10" refY="3.5" orient="auto" fill="#555">
    <path d='M 0,-5 L 10 ,0 L 0,5' />
  </marker>
  </marker>
        </defs>
      
      
      </svg>
      <script src="https://d3js.org/d3.v5.min.js"></script>
      <script>
              d3.select("svg").selectAll("*").remove();
         // Load the network graph
          var graph = JSON.parse('{graph_json}'); 

         
 
     
         var nodes = graph.nodes;
         var links = graph.links;

         var svg = d3.select("svg"),
             width = +svg.attr("width"),
             height = +svg.attr("height");

        var centralityScale = d3.scaleLinear()
         .domain([d3.min(nodes, d => d.centrality), d3.max(nodes, d => d.centrality)])
         .range([5, 35]);  // Min and max circle radii
         
        var fontSizeScale = d3.scaleLinear()
        .domain([d3.min(nodes, d => d.centrality), d3.max(nodes, d => d.centrality)])
        .range([10, 30]);  // Min and max font sizes, adjust as needed
        
        var opacityScale = d3.scaleLinear()
        .domain([d3.max(nodes, d => d.centrality), d3.min(nodes, d => d.centrality)])
        .range([0.2, 0.7]);  // Min and max font sizes, adjust as needed

function getYearPeriod(year, timeSpan) {{
    var periodStart = year - (year % timeSpan);
    var periodEnd = periodStart + timeSpan - 1;
    return `${{periodStart}}-${{periodEnd}}`;
}}

function createYearAnchors(startYear, endYear, timeSpan, canvasHeight) {{
    var yearAnchors = {{}};
    var numberOfPeriods = Math.ceil((endYear - startYear + 1) / timeSpan);
    var periodHeight = canvasHeight / numberOfPeriods;
    var yOffset = periodHeight / 2;

    for (var year = startYear; year <= endYear; year += timeSpan) {{
        var period = getYearPeriod(year, timeSpan);
        yearAnchors[period] = {{
            x: 500,  // Central x-axis position, adjust as needed
            y: yOffset + ((year - startYear) / timeSpan) * periodHeight
        }};
    }}

    return yearAnchors;
}}

var yearAnchors = createYearAnchors(1950, 2020, 5, 4000);  // Example usage


var simulation = d3.forceSimulation(nodes)
    .force("link", d3.forceLink(links).id(d => d.id))
    .force("charge", d3.forceManyBody().strength(-150))
    .force("center", d3.forceCenter(width / 2, height / 2))
    .force("yearAnchor", alpha => forceYearAnchor(alpha, nodes, yearAnchors, 5)); // Pass nodes directly

function forceYearAnchor(alpha, nodes, yearAnchors, timeSpan) {{
    var strength = 0.1;
    nodes.forEach(function(d) {{
        var period = getYearPeriod(d.year, timeSpan);
        var target = yearAnchors[period];
        if (!target) {{
            console.warn('No target found for period:', period);
            return;
        }}
        d.vx += (target.x - d.x) * alpha * strength;
        d.vy += (target.y - d.y) * alpha * strength;
    }});
}}

                 
         // Add links
 var link = svg.append("g")
          .attr("class", "links")
          .selectAll("path")
          .data(links)
          .enter().append("path")
          .attr("fill", "none")
          .attr("stroke", "#555")
          .attr("stroke-width", 0.8)
          .attr("stroke-opacity", 0.2)
          .attr("marker-end", "url(#arrowhead)");  
         
         // Add nodes
         // Create groups for nodes and labels
         var node = svg.append("g")
         .attr("class", "nodes")
         .selectAll(".node")
         .data(nodes)
         .enter().append("g")
         .attr("class", "node")
         .call(d3.drag()
         .on("start", dragstarted)
         .on("drag", dragged)
         .on("end", dragended));
         
         // Append circles to the groups
         var color = d3.scaleOrdinal(d3.schemeCategory10);  // adjust if needed
         
         // When appending circles:
         node.append("circle")
         .attr("r", d => centralityScale(d.centrality))
         .attr("fill", d => color(d.community))
         .attr("fill-opacity", d => opacityScale(d.centrality));
         
         
         // Append labels to the groups
        node.append("text")
        .attr("dx", 12)
        .attr("dy", ".35em")
        .style("font-size", d => fontSizeScale(d.centrality) + "px")
        .style("fill", "darkslategray")
        .text(function(d) {{
            return d.label;
        }});
         

         
         
         simulation
         .nodes(nodes)
         .on("tick", ticked);
         
         simulation.force("link")
         .links(links);
         
// Function to highlight a cluster
function highlightCluster(clusterId, enlarge, nodeColor) {{
    node.selectAll("circle")
        .filter(function(d) {{ return d.community === clusterId; }})
        .transition()
        .duration(150)
        .attr("r", function(d) {{ 
            return enlarge ? centralityScale(d.centrality) * 1.5 : centralityScale(d.centrality);
        }});

    // Also highlight the arcs
    link.transition()
        .duration(150)
        .attr("stroke", function(l) {{
            return (l.source.community === clusterId || l.target.community === clusterId) 
                ? nodeColor // Highlight color
                : "#555";  // Default color
        }})
        .attr("stroke-opacity", function(l) {{
            return (l.source.community === clusterId || l.target.community === clusterId) 
                ? 0.9 // Highlight opacity
                : 0.2;  // Default opacity
        }});
}}

node.on("mouseover", function(d) {{
    var nodeColor = color(d.community); // Retrieve the color of the hovered node
    highlightCluster(d.community, true, nodeColor);
    // Optional: Display a tooltip or additional info
    // ... tooltip code ...
}})
.on("mouseout", function(d) {{
    // Reset nodes and links to their default state
    node.selectAll("circle")
        .transition()
        .duration(150)
        .attr("r", d => centralityScale(d.centrality));

    link.transition()
        .duration(150)
        .attr("stroke", "#555")  // Default link color
        .attr("stroke-opacity", 0.2);  // Default link opacity

    // Optional: Hide the tooltip
    // ... tooltip code ...
}});


         
         // Add node labels
         
         
         // ... rest of the code ...
                     
function ticked() {{
    node.attr("transform", d => `translate(${{d.x}},${{d.y}})`);

    link.attr("d", function(d) {{
        var dx = d.target.x - d.source.x,
            dy = d.target.y - d.source.y,
            dr = Math.sqrt(dx * dx + dy * dy),
            rSource = centralityScale(d.source.centrality), // Radius of source node
            rTarget = centralityScale(d.target.centrality), // Radius of target node
            offsetXSource = (dx * rSource) / dr,
            offsetYSource = (dy * rSource) / dr,
            offsetXTarget = (dx * rTarget) / dr,
            offsetYTarget = (dy * rTarget) / dr;

        // Control the amount of arching
        var arcScale = 0.5; // Adjust this value to increase or decrease arching
        var arcRadius = dr / arcScale;

        var arcPath = `M ${{d.source.x + offsetXSource}} ${{d.source.y + offsetYSource}} 
                       A ${{arcRadius}} ${{arcRadius}} 0 0 1 
                       ${{d.target.x - offsetXTarget}} ${{d.target.y - offsetYTarget}}`;
        return arcPath;
    }});
}}

         
         function dragstarted(d) {{
         if (!d3.event.active) simulation.alphaTarget(0.3).restart();
         d.fx = d.x;
         d.fy = d.y;
         }}
         
         function dragged(d) {{
         d.fx = d3.event.x;
         d.fy = d3.event.y;
         }}
         
         function dragended(d) {{
         if (!d3.event.active) simulation.alphaTarget(0);
         d.fx = null;
         d.fy = null;
         }}
         
      </script>
   </body>
</html>""")
